In [ ]:
# Analys av en tråd på Flashback

In [ ]:
!python3 -m pip install --upgrade matplotlib networkx pandas pyvipr > /dev/null
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager > /dev/null
#!jupyter labextension install pyvipr > /dev/null
!jupyter nbextension enable --py --sys-prefix pyvipr

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
# Definiera tråd och användare du är intresserad av
thread = ""
user = ""

In [ ]:
# Variabler 
database = 'flashbackscraper/' + thread + ".sqlite3"
gexf = 'flashbackscraper/' + thread + ".gexf"
gexf_image = 'flashbackscraper/' + thread + ".png"

In [ ]:
![[ ! -d flashbackscraper ]] && git clone https://github.com/christopherkullenberg/flashbackscraper.git
!python3 -m pip --quiet install beautifulsoup4 lxml requests[socks]
![[ ! -e flashbackscraper/{thread}.sqlite3 ]] && cd flashbackscraper && python3 flashbackscraper.py -u "https://www.flashback.org/{thread}"

In [ ]:
# Öppna en connection till databasen från flashbackscaper
conn = sqlite3.connect(database)

In [ ]:
# Läs in sqlite-database och spara den i en dataframe
flashback_df = pd.read_sql_query("SELECT * FROM fb", conn)

In [ ]:
print("Det finns", str(len(flashback_df)), "inlägg från tråden på Flashback i den inlästa databasen.") 

In [ ]:
# Specifiera tidsformat i DataFrame.
flashback_df.time = pd.to_datetime(flashback_df.date + flashback_df.time, format='%Y-%m-%d%H:%M')
flashback_df.date = pd.to_datetime(flashback_df.date, format='%Y-%m-%d')

In [ ]:
# Visa alla inlägg.
pd.set_option('display.max_rows', None) # pd.reset_option('display.max_rows') to reset
pd.set_option('max_colwidth', None)
flashback_df

In [ ]:
# Vilka användare är aktiva i tråden?
flashback_df['user'].value_counts()

In [ ]:
# Titta närmare på en användare och spara informationen i en dataframe user_df
user_df = pd.read_sql_query("SELECT * FROM fb WHERE user = '" + user + "'", conn)
print(str(len(user_df)), "data points.")
# Lägg till fält med aktiv timme
user_df['hour'] = user_df.time.str[:2]
user_df.time = pd.to_datetime(user_df.date + user_df.time, format='%Y-%m-%d%H:%M')
user_df.date = pd.to_datetime(user_df.date, format='%Y-%m-%d')

In [ ]:
procent = len(user_df)/len(flashback_df)*100
print("Av alla %s inlägg i tråden är %s skrivna av användare %s vilket motsvarar %2.2f%%." % (str(len(flashback_df)), str(len(user_df)), user, procent))

In [ ]:
# Visa utvalda kolumner användaren
user_df.iloc[:,[0,4,5,6,9]]

In [ ]:
# Vilka inlägg är svar på andras inlägg?
# none nedan indikerar inlägg skrivna utan att vara svar.
# Siffran anger antal svar till olika andra användare.
user_df['inreply'].value_counts()

In [ ]:
# Sedan tidigare finns en kolumn "hour" i DataFrame wb_df med den timme som ett inlägg är gjort.
# Använd .value_counts() för att räkna ut antalet.
print(user_df['hour'].value_counts())

In [ ]:
# Plotta aktiva timmar över dygnets timmar.
# Ska en ny DataFrame hours_df med bara timmar och antal.
hours_df = user_df['hour'].value_counts().rename_axis('unique_values').reset_index(name='counts')
hours_df['unique_values'] = pd.to_numeric(hours_df['unique_values'])

In [ ]:
# Rita graf
hours = [0] * 24
for nr, row in hours_df.iterrows():
    hours[row[0]] = row[1]

i = 0
x = []
while i < 24:
    x.append(i)
    i=i+1

plt.xticks(x, x)
plt.bar(x, hours, width=1, align='edge')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
# Sätt en cursor till databasen
db = sqlite3.connect(database)
cursor = db.cursor()

In [ ]:
# Läs in alla kolumnerna user och inreply
selectedges = cursor.execute('SELECT user, inreply FROM fb')
edges = cursor.fetchall()

In [ ]:
# Skapa en tom graf
G = nx.Graph()

In [ ]:
# Lägg in kopplingar mellan inlägg
for e in edges:
    if e[1] == 'none':
         continue
    else:
         G.add_edge(e[0], e[1])

In [ ]:
# Skapa fast bild för utskrifter.
fig = plt.figure(figsize=(19,13))
ax = plt.subplot(111)
ax.set_title('Kopplingar mellan användare', fontsize=25)

pos = nx.shell_layout(G)
nx.draw(G, pos, node_size=1500, node_color='red', font_size=10, font_weight='bold')
nx.draw_networkx_nodes(G, pos, node_size=10, node_color="w", alpha=0.4)
nx.draw_networkx_edges(G, pos, alpha=0.4, node_size=0, width=1, edge_color="k")
nx.draw_networkx_labels(G, pos, font_size=18)
plt.tight_layout()
plt.savefig(gexf_image, format="PNG")
plt.show()